In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt

import scipy as sp
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import utils
from sklearn import metrics

# Testing OneClassSVM

In [3]:
#unsupervised outlier detection
#anomaly detection
#train only on protons

In [4]:
p_data = sp.sparse.load_npz('../data/tilt/20x20x20/pDisc_40000_20x20x20_tilt.npz')
p_labels = np.ones((p_data.shape[0],))

In [5]:
p_train, p_test, p_labels_train, p_labels_test = train_test_split(p_data, p_labels, test_size=0.25, random_state=42)

In [6]:
#import carbon and noise for test
C_data = sp.sparse.load_npz('../data/tilt/20x20x20/CDisc_40000_20x20x20_tilt.npz')
noise_data = sp.sparse.load_npz('../data/tilt/20x20x20/noiseDisc_40000_20x20x20.npz')

In [7]:
print(C_data.shape)
print(noise_data.shape)

C_test = C_data[0:10000]
noise_test  = noise_data[0:10000]

print(C_test.shape)
print(noise_test.shape)

(40001, 8000)
(40000, 8000)
(10000, 8000)
(10000, 8000)


In [16]:
full_test = sp.sparse.vstack([p_test, C_test, noise_test], format='csr')

#third label array p, c, j

full_labels = np.hstack((p_labels_test, np.negative(np.ones(C_test.shape[0])), np.negative(np.ones(noise_test.shape[0]))))
#print(full_data_nocharge.shape)

full_test, full_labels = utils.shuffle(full_test, full_labels)

print(full_test.shape)
print(full_labels.shape)

(30001, 8000)
(30001,)


In [28]:
clf = svm.OneClassSVM(nu=0.05, kernel='rbf')
clf.fit(p_train[0:10000])

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto', kernel='rbf',
      max_iter=-1, nu=0.05, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

In [29]:
subset_size = 5000

In [30]:
y_pred_train = clf.predict(p_train[0:subset_size]) 
y_pred_test = clf.predict(full_test)

In [31]:
print("f1-score with proton training data: " + str(metrics.f1_score(p_labels_train[0:subset_size], y_pred_train)))
print("f1-score with mixed proton/carbon/noise test data: " + str(metrics.f1_score(full_labels, y_pred_test)))

f1-score with proton training data: 0.973938025857
f1-score with mixed proton/carbon/noise test data: 0.57096851145


In [ ]:
print(p_labels_train[0:subset_size])
print(y_pred_train)